In [ ]:
import os
import cv2
from ultralytics import YOLO
import mediapipe as mp
import time

# === 設定路徑 ===
YOLO_DISPENSER_MODEL = "/home/jetson/projects/weights/best.engine"
YOLO_PEOPLE_MODEL = "/home/jetson/projects/weights/yolov8n.engine"
VIDEO_INPUT = "/home/jetson/projects/videos/input.mp4"
VIDEO_OUTPUT = "/home/jetson/projects/videos/output.mp4"

# === Mediapipe 初始化 ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.3, min_tracking_confidence=0.3)
mp_draw = mp.solutions.drawing_utils

# === 載入 YOLO TensorRT 模型 ===
model_people = YOLO(YOLO_PEOPLE_MODEL)
model_dispenser = YOLO(YOLO_DISPENSER_MODEL)

# === 全域變數 ===
dispenser_roi = None
sanitized_count = 0
sanitized_ids = set()
track_state = {}
INTERSECTION_THRESHOLD = 0.1
DELAY_TIME = 3
HAND_CHECK_INTERVAL = 5  # 每 5 幀檢查一次

# === 初始化影片 ===
def initialize_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    return cap, out, fps, frame_width, frame_height

# === 偵測酒精機位置 ===
def detect_dispenser(frame):
    results = model_dispenser(frame)
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls.item())
            label = r.names[cls_id]
            confidence = box.conf[0]
            if label.lower() == "dispenser" and confidence > 0.3:
                print(f"Detected dispenser at: {box.xyxy[0].tolist()}")
                return list(map(int, box.xyxy[0]))
    print("No dispenser detected.")
    return None

# === 計算重疊比例 ===
def calculate_intersection_ratio(person_box, dispenser_roi):
    def area(box):
        return max(0, box[2] - box[0]) * max(0, box[3] - box[1])

    x1 = max(person_box[0], dispenser_roi[0])
    y1 = max(person_box[1], dispenser_roi[1])
    x2 = min(person_box[2], dispenser_roi[2])
    y2 = min(person_box[3], dispenser_roi[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    return inter_area / min(area(person_box), area(dispenser_roi))

# === 優化版手部偵測（降頻 + 裁切）===
def process_hand_detection_lite(frame, dispenser_roi, track_id, w, h, frame_count):
    global sanitized_count
    if frame_count % HAND_CHECK_INTERVAL != 0:
        return False

    roi_frame = frame[dispenser_roi[1]:dispenser_roi[3], dispenser_roi[0]:dispenser_roi[2]]
    rgb = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    if results.multi_hand_landmarks:
        for hand in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(roi_frame, hand, mp_hands.HAND_CONNECTIONS)
            lower_start = int((dispenser_roi[3] - dispenser_roi[1]) * 0.8)
            lower_end = dispenser_roi[3] - dispenser_roi[1]
            for lm in hand.landmark:
                x = int(lm.x * (dispenser_roi[2] - dispenser_roi[0]))
                y = int(lm.y * (dispenser_roi[3] - dispenser_roi[1]))
                if 0 <= x <= (dispenser_roi[2] - dispenser_roi[0]) and lower_start <= y <= lower_end:
                    if track_id not in sanitized_ids:
                        sanitized_ids.add(track_id)
                        sanitized_count += 1
                        return True
    return False

# === 處理追蹤與判斷消毒 ===
def process_tracking_optimized(frame, results_people, dispenser_roi, w, h, current_time, frame_count):
    for r in results_people:
        for box, track_id_tensor in zip(r.boxes, r.boxes.id):
            track_id = int(track_id_tensor.item())
            cls_id = int(box.cls.item())
            label = r.names[cls_id]
            if label != "person":
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            person_box = (x1, y1, x2, y2)
            track_state.setdefault(track_id, {'last_detected': 0, 'show_text': False})

            overlap_ratio = calculate_intersection_ratio(person_box, dispenser_roi)
            if overlap_ratio > INTERSECTION_THRESHOLD:
                if (current_time - track_state[track_id]['last_detected']) > DELAY_TIME:
                    if process_hand_detection_lite(frame, dispenser_roi, track_id, w, h, frame_count):
                        track_state[track_id] = {'last_detected': current_time, 'show_text': True}

            if track_state[track_id]['show_text']:
                cv2.putText(frame, "Sanitization Success!", (w // 4, h // 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# === 主程式開始 ===
cap, out, fps, frame_width, frame_height = initialize_video(VIDEO_INPUT, VIDEO_OUTPUT)

# 初始化 dispenser ROI
while dispenser_roi is None and cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    dispenser_roi = detect_dispenser(frame)

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("處理完成")
        break

    current_time = time.time()
    h, w, _ = frame.shape

    results_people = model_people.track(frame, persist=True, tracker="botsort.yaml")
    process_tracking_optimized(frame, results_people, dispenser_roi, w, h, current_time, frame_count)

    cv2.putText(frame, f"Sanitized Count: {sanitized_count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()